In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pyfmi import load_fmu
import pyfmi
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyfmi import load_fmu
import pyfmi
import numpy as np
import joblib
from tensorflow import keras
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import mean_squared_error
import math

In [2]:
# 初始化标志变量
params_updated = False
saved_number = None
Need_change = False

a = 0
stop_t = 86400 * 33  # 运行33天
step_size = 60
model1 = load_fmu("Morning_start_compared2.fmu")
model1.initialize()

# 初始化数据存储
cool_load_total_data = []
cool_load_store_data = []
Energy_con = []
outdoor_temperature_data = []
indoor_temperature_checkpoint = []
indoor_temperature_data = []  # 存储重命名的indoor_temperature
chiller_number_data = []      # 存储重命名的chiller_number
time_data = []                # 存储时间数据




# 星期映射表，0代表星期四，1代表星期五，依此类推
week_days = [4,5,6,7,1,2,3]

# 初始天数偏移，确保第一天为星期四
initial_day_offset = 0  # 因为第0天是星期四


while a < stop_t:
    status = model1.do_step(a, step_size)
    if status == pyfmi.fmi.FMI_OK:
  
        ## make sure the time
        date_update = a % 86400
        current_day = (a // 86400 + initial_day_offset) % 7  # 当前是第几天
        week_day = week_days[current_day]  # 当前是星期

        ## get the outdoor temperature, TEVA, TCON, indoor temperature at 7:00 AM
        if date_update == 21600:  # 21600秒 = 6小时 * 3600秒/小时
            outdoor_temperature = model1.get('room_cooling_model.timTab5.y[1]')
            outdoor_temperature = outdoor_temperature[0]-273.15
            outdoor_temperature_data.append(outdoor_temperature)
            chiller_EVA_temperature = model1.get('mulChi.chi[1].TEvaEnt')
            chiller_CON_temperature = model1.get('mulChi.chi[1].TConEnt')
            indoor_temperature = model1.get('room_cooling_model.T_room')
            indoor_temperature = indoor_temperature[0]
            if week_day < 6:
                if Need_change:
                    chiller_number = 3.5
                else:
                    chiller_number = 3
            else:
                if Need_change:
                    chiller_number = 2.5
                else:
                    chiller_number = 2                
                
                
            model1.set('supper_chiller_number_control.chillern', chiller_number)
        # save the optimal value
            saved_number = chiller_number                
            params_updated = True
            
        ## keep the optimal value until the next change
        if params_updated:
            model1.set('supper_chiller_number_control.chillern', saved_number)            
         
        
        # 从早上8点到早上10点提取indoor_temperature和chiller_number
        if 25200 <= date_update <= 36000:
            indoor_temperature_renamed = model1.get('room_cooling_model.T_room')
            indoor_temperature_data.append(indoor_temperature_renamed[0] - 273.15)
            chiller_number = model1.get('chiller_number_control.n')
            chiller_number_data.append(chiller_number[0])
            # 提取时间并格式化为HH:MM
            hours = (a // 3600) % 24
            minutes = (a // 60) % 60
            time_formatted = f"{hours:02}:{minutes:02}"
            time_data.append(time_formatted)
        
               
        
        ## check the room temperature at 9:00
        if date_update == 32400:
            room_temperature_check = model1.get('room_cooling_model.T_room')
            indoor_temperature_checkpoint.append(room_temperature_check[0] - 273.15)
            if room_temperature_check[0] - 273.15 > 24:
                Need_change = True
            else:
                Need_change = False
                
        ## get the cooling load data and consumption data at noon (12:00)
        if date_update == 43200:
            cooling_all = model1.get("Cooling_load1.y")
            cooling_single = model1.get("Cooling_load.y")
            energy_consumption = model1.get("intWitRes1.y")
            cool_load_total_data.append(cooling_all[0]/60)
            cool_load_store_data.append(cooling_single[0] / 3600)
            Energy_con.append(energy_consumption[0])             
        
        a = a + step_size
    else:
        print("error")
        break           

In [3]:
check_show619 = pd.DataFrame({"indoor_temperature":indoor_temperature_data,'time':time_data,
                              "chiller_number":chiller_number_data})

In [4]:
check_show619

,indoor_temperature,time,chiller_number
0,28.025948,07:00,3.0
1,27.832315,07:01,3.0
2,27.630960,07:02,3.0
3,27.436452,07:03,3.0
4,27.249120,07:04,3.0
...,...,...,...
5968,24.258149,09:56,3.0
5969,24.252777,09:57,3.0
5970,24.246860,09:58,3.0
5971,24.239901,09:59,3.0


In [5]:
check_show619.to_csv('room_temperature_number619.csv')

In [4]:
# 删除列表的前三项
indoor_temperature = indoor_temperature_checkpoint[3:]
Energy_consumption = Energy_con[3:]
cool_load_total = cool_load_total_data[3:]
cool_load_store = cool_load_store_data[3:]
outdoor_temperature = outdoor_temperature_data[3:]

In [5]:
result_show = pd.DataFrame({"indoor_temperature":indoor_temperature,"outdoor_temperature":outdoor_temperature,
                            "energy_consumption":Energy_consumption,"cooling_load_total":cool_load_total,"cooling_load_11":cool_load_store})

In [12]:
optimal_result = pd.read_csv('optimal_reulst_morning_start.csv')

In [10]:
result_show.to_csv('morning_start_compared.csv')

In [14]:
result_show = pd.read_csv('morning_start_compared.csv')


,Date,indoor_temperature_x,outdoor_temperature_x,optimal_chiller,optimal_time,energy_consumption_x,cooling_load_total_x,cooling_load_11_x,indoor_temperature_y,outdoor_temperature_y,energy_consumption_y,cooling_load_total_y,cooling_load_11_y
